In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.impute import SimpleImputer

# 1. 데이터 로드 및 전처리
def load_and_preprocess(file_path_user, file_paths_restaurant):
    """
    사용자와 식당 데이터를 로드하고 전처리합니다.

    Parameters:
        file_path_user (str): 사용자 데이터 파일 경로.
        file_paths_restaurant (list): 식당 데이터 파일 경로 리스트.

    Returns:
        tuple: 전처리된 사용자 데이터, 식당 데이터, 사용자 ID, 식당 ID, 통합 식당 데이터
    """
    user_data = pd.read_csv(file_path_user)

    # 식당 데이터 통합
    restaurant_data_list = [pd.read_csv(file_path) for file_path in file_paths_restaurant]
    restaurant_data = pd.concat(restaurant_data_list, ignore_index=True)

    # 사용자와 식당의 ID 추출
    user_ids = user_data['nickname'].tolist()
    restaurant_ids = restaurant_data['가게명'].tolist()

    # 숫자형 데이터 추출
    user_data_numeric = user_data.select_dtypes(include=['float64', 'int64'])
    restaurant_data_numeric = restaurant_data.select_dtypes(include=['float64', 'int64'])

    # 결측치 처리
    imputer = SimpleImputer(strategy='mean')
    user_data_imputed = imputer.fit_transform(user_data_numeric)
    restaurant_data_imputed = imputer.fit_transform(restaurant_data_numeric)

    # 비율 계산 (빈도수를 전체 합으로 나눔)
    restaurant_data_proportions = restaurant_data_imputed / restaurant_data_imputed.sum(axis=1, keepdims=True)

    # 정규화
    scaler = StandardScaler()
    user_data_scaled = scaler.fit_transform(user_data_imputed)
    restaurant_data_scaled = scaler.fit_transform(restaurant_data_proportions)

    return user_data_scaled, restaurant_data_scaled, user_ids, restaurant_ids, restaurant_data

# 2. PCA 수행
def perform_pca(data, n_components=3):
    """
    입력 데이터를 주성분으로 변환합니다.

    Parameters:
        data (ndarray): 입력 데이터.
        n_components (int): 주성분 개수.

    Returns:
        ndarray: 축소된 데이터
    """
    pca = PCA(n_components=n_components)
    return pca.fit_transform(data)

# 3. 유사도 계산
def calculate_recommendation_scores(user_features, restaurant_features):
    """
    사용자와 식당 데이터 간 코사인 유사도를 계산합니다.

    Parameters:
        user_features (ndarray): 사용자 데이터.
        restaurant_features (ndarray): 식당 데이터.

    Returns:
        ndarray: 유사도 점수 배열
    """
    return cosine_similarity(user_features, restaurant_features)

# 4. 추천 리스트 생성
def generate_recommendations(scores, user_ids, restaurant_ids, top_n=5):
    """
    사용자별 상위 N개의 추천 식당을 생성합니다.

    Parameters:
        scores (ndarray): 유사도 점수 배열.
        user_ids (list): 사용자 ID 리스트.
        restaurant_ids (list): 식당 ID 리스트.
        top_n (int): 추천할 식당 개수.

    Returns:
        DataFrame: 추천 결과
    """
    recommendations = []
    for i, user_id in enumerate(user_ids):
        top_indices = scores[i].argsort()[::-1][:top_n]
        top_restaurants = [restaurant_ids[j] for j in top_indices]
        top_scores = [scores[i][j] for j in top_indices]
        recommendations.append({"user_id": user_id, "recommended_restaurants": top_restaurants, "scores": top_scores})
    return pd.DataFrame(recommendations)

# 5. 결과 출력 함수
def print_recommendations(recommendations):
    """
    추천 결과를 출력합니다.

    Parameters:
        recommendations (DataFrame): 추천 결과 데이터프레임
    """
    for _, row in recommendations.iterrows():
        print(f"User ID: {row['user_id']}")
        print("Recommended Restaurants and Scores:")
        for restaurant, score in zip(row['recommended_restaurants'], row['scores']):
            print(f"  - {restaurant}: {score:.4f}")
        print()

# 6. 전체 파이프라인
def main(file_path_user, file_paths_restaurant, n_components=3, top_n=5):
    """
    PCA 기반 추천 시스템 실행.

    Parameters:
        file_path_user (str): 사용자 데이터 파일 경로.
        file_paths_restaurant (list): 식당 데이터 파일 경로 리스트.
        n_components (int): PCA 주성분 개수.
        top_n (int): 추천할 식당 개수.
    """
    # 데이터 로드 및 전처리
    user_data, restaurant_data, user_ids, restaurant_ids, _ = load_and_preprocess(
        file_path_user, file_paths_restaurant
    )

    # PCA 및 유사도 계산
    user_features = perform_pca(user_data, n_components)
    restaurant_features = perform_pca(restaurant_data, n_components)
    scores = calculate_recommendation_scores(user_features, restaurant_features)

    # 추천 결과 생성 및 출력
    recommendations = generate_recommendations(scores, user_ids, restaurant_ids, top_n)
    print_recommendations(recommendations)

# 실행
user_file_path = "aggregated_final_result.csv"
restaurant_file_paths = [
    "df_cafe.csv",
    "df_drink.csv",
    "df_meal.csv"
]
main(user_file_path, restaurant_file_paths, n_components=3, top_n=5)

User ID: 아암마시따
Recommended Restaurants and Scores:
  - 소소라면 닭꼬치: 0.9895
  - 베나레스 숙대점: 0.9814
  - 더블랙웨일바: 0.9553
  - 탄막: 0.9548
  - 모토카레시: 0.9526

User ID: 밍닝몽
Recommended Restaurants and Scores:
  - 대관령목장: 0.9996
  - 김밥2000: 0.9960
  - 두리식당: 0.9943
  - 철길우동까스: 0.9857
  - 교촌치킨 숙대점: 0.9850

User ID: 후암동 올리
Recommended Restaurants and Scores:
  - 앙꼬: 0.8815
  - 쥬케로카페 숙대점: 0.8748
  - 가든한밤: 0.8144
  - 스타벅스 숙대점: 0.7946
  - 카페나리나무: 0.7830

User ID: Junns
Recommended Restaurants and Scores:
  - 쥬케로카페 숙대점: 0.9958
  - 스타벅스 숙대점: 0.9957
  - 스택빈 숙대점: 0.9942
  - 앙꼬: 0.9898
  - 카페 퓨엔테: 0.9786

User ID: whrhwhdoo
Recommended Restaurants and Scores:
  - 타코비 숙명여대점: 0.9996
  - 마포구이마당 숙대직영점: 0.9495
  - 홍곱창 숙명여대점: 0.8955
  - 홍짜장 숙대점: 0.8641
  - 별진화로구이: 0.7919

User ID: 뚝딱64
Recommended Restaurants and Scores:
  - 홍곱창 숙명여대점: 0.8325
  - 몬스터플레이스: 0.8191
  - 평화남영: 0.7192
  - 소소라면 닭꼬치: 0.6864
  - 이웃집영준이: 0.6577

User ID: 잡덕수집가
Recommended Restaurants and Scores:
  - 별진화로구이: 0.9568
  - 타코비 숙명여대점: 0.9234
  - 홍짜장 